<a href="https://colab.research.google.com/github/IsaacBrrP/ARLETEX/blob/main/ingEst_ventasARLETEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# ============================================================
# PASO 1: Carga robusta del CSV
# ============================================================
# El CSV tiene números con comas dentro de comillas ("2,460.00")
# pandas puede manejar esto con thousands=','

df = pd.read_csv(
    'CopiaProductos Vendidos_Limpia.csv',
    thousands=',',           # Para parsear "2,460.00" → 2460.00
    encoding='utf-8'         # Ajustar si hay problemas con acentos
)

print(f"Shape: {df.shape}")
print(f"Columnas: {df.columns.tolist()}")
print(f"\nPrimeras filas:\n{df.head()}")
print(f"\nÚltimas filas:\n{df.tail()}")
print(f"\nTipos de datos:\n{df.dtypes}")

Shape: (20555, 39)
Columnas: ['ID', 'Fecha_Ini', 'Fecha_Fin', 'Estatus', 'Id_Producto', 'Producto', 'Descripcion_Producto', 'Unidad', 'Cant', 'Cant_Por_Entregar', 'Cant_Entregada', 'PU', 'Costo_Unitario', 'Utilidad', 'Utilidad%', 'Subtotal', 'IVA', 'Total', 'Origen', 'ID_Color', 'Color', 'Clave', 'Nivel1', 'Nivel2', 'Nivel3', 'Tipo_Envio', 'Id_Paqueteria', 'Paqueteria', 'No_Guia', 'Fecha_Entrega', 'Id_Cliente', 'ENTR_CALLE', 'ENTR_EXT', 'ENTR_INT', 'ENTR_COLONIA', 'ENTR_CIUDAD', 'ENTR_CP', 'ENTR_ESTADO', 'ENTR_PAIS']

Primeras filas:
  ID  Fecha_Ini  Fecha_Fin    Estatus Id_Producto            Producto  \
0  1   7/4/2022   7/4/2022  ENTREGADO        3207         Aida 150-11   
1  2   7/4/2022   7/4/2022  ENTREGADO        3205  Aida Torzal 150/11   
2  3  5/23/2022  5/23/2022  ENTREGADO        1387          Back Satin   
3  4   6/1/2022   6/1/2022  ENTREGADO        1380          Back Satin   
4  5   6/1/2022   6/1/2022  ENTREGADO        1406          Back Satin   

                     

/tmp/ipython-input-279/4246955001.py:10: DtypeWarning: Columns (0,4,8,9,10,11,12,13,14,15,16,17,19,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [2]:
# ============================================================
# PASO 2: Perfilado general de calidad
# ============================================================

def perfilado_calidad(df):
    """Genera un reporte de calidad por columna"""
    report = pd.DataFrame({
        'tipo_dato': df.dtypes,
        'no_nulos': df.count(),
        'nulos': df.isnull().sum(),
        'pct_nulos': (df.isnull().sum() / len(df) * 100).round(2),
        'unicos': df.nunique(),
        'pct_unicos': (df.nunique() / len(df) * 100).round(2),
        'ejemplo_1': df.iloc[0] if len(df) > 0 else None,
        'ejemplo_2': df.iloc[10] if len(df) > 10 else None,
    })
    return report

print(perfilado_calidad(df).to_string())

                     tipo_dato  no_nulos  nulos  pct_nulos  unicos  pct_unicos                                                                                                                                     ejemplo_1                                     ejemplo_2
ID                      object     20551      4       0.02   20551       99.98                                                                                                                                             1                                            11
Fecha_Ini               object     20551      4       0.02     704        3.42                                                                                                                                      7/4/2022                                     2/21/2022
Fecha_Fin               object     20551      4       0.02     704        3.42                                                                                                                         

In [3]:
# ============================================================
# PASO 4: Parseo de fechas y validación temporal
# ============================================================

df['Fecha_Ini'] = pd.to_datetime(df['Fecha_Ini'], format='%m/%d/%Y', errors='coerce')
df['Fecha_Fin'] = pd.to_datetime(df['Fecha_Fin'], format='%m/%d/%Y', errors='coerce')

# Fechas que no parsearon
print(f"Fechas inicio no parseadas: {df['Fecha_Ini'].isna().sum()}")
print(f"Fechas fin no parseadas:    {df['Fecha_Fin'].isna().sum()}")

# Rango temporal
print(f"\nRango Fecha_Ini: {df['Fecha_Ini'].min()} → {df['Fecha_Ini'].max()}")
print(f"Rango Fecha_Fin: {df['Fecha_Fin'].min()} → {df['Fecha_Fin'].max()}")

# Lead time (diferencia entre inicio y fin)
df['lead_time_dias'] = (df['Fecha_Fin'] - df['Fecha_Ini']).dt.days
print(f"\nLead time (días):")
print(df['lead_time_dias'].describe())

Fechas inicio no parseadas: 6
Fechas fin no parseadas:    6

Rango Fecha_Ini: 2022-01-03 00:00:00 → 2024-08-20 00:00:00
Rango Fecha_Fin: 2022-01-03 00:00:00 → 2024-08-20 00:00:00

Lead time (días):
count    20549.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: lead_time_dias, dtype: float64
